In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.read_excel(io = '####.xlsx', sheet_name = 'Load')

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
def campus_metrics(df,campus_list,year):
    campus_metrics_dict = {}
    for campus in campus_list:
        filtered_df = df[(df['campus']==campus)&(df['service_date'].dt.year==year)]
        attendance = filtered_df['attendance'].sum()
        baptisms = filtered_df['baptisms'].sum()
        campus_metrics_dict[campus] = {'attendance':attendance,'baptisms':baptisms}
    return campus_metrics_dict

In [ ]:
campus_list = df['campus'].value_counts().index.to_list()
print(campus_list)

In [ ]:
metrics_2018 = campus_metrics(df,campus_list,2018)


In [ ]:
print(metrics_2018)

In [ ]:
import pprint as pp
printer = pp.PrettyPrinter(indent=4)
printer.pprint(metrics_2018)


In [ ]:
df[df['Unnamed: 10']==7].head()

In [ ]:
#Unnamed column contains whether the day was a Saturday, Sunday or another day. This is good for my calculations.
#let's create a function that assigns specific week of the year to each row and for days other than Sunday, assigns to the week of the following sunday

attendance_df = df.copy()
attendance_df.rename(columns={'Unnamed: 10':'dow'},inplace=True)
attendance_df.drop(labels='timezone',axis=1,inplace=True)

In [ ]:
def return_WOY(series):
    woy = series.dt.week
    return woy

In [ ]:
attendance_df['woy'] = attendance_df['service_date'].agg(return_WOY)
attendance_df[attendance_df['service_date'].dt.month.isin([4,12])]

In [ ]:
def adjust_woy(df):
    for i in range(o,df.shape[0]):
        dow = df.loc[i,'dow']
        woy = df.loc[i,'woy']
        try:
            ndow = df.loc[i-1,'dow']
            


## Researching CSV calculations Vs Fact Metrics:


In [ ]:
df['Year']=df['service_date'].dt.year #adding a year and month column to ease grouping
df['Month']=df['service_date'].dt.month
df.head()

In [ ]:
#grouping by Year
df_by_year = df[df['Year'].isin([2017,2018,2019])].groupby(['campus','Year'],as_index=False)[['attendance','baptisms','salvations']].sum()

In [ ]:
pd.set_option("display.max_rows",1000)
df_by_year

## Bringing warehouse data into dataframe for faster comparisson
Rather than checking manually between the csv and the warehouse, I'll just connect to the warehouse via python sqlalchemy and compare the two dataframes. This will make the work faster for other comparissons as well

In [ ]:
import sqlalchemy as db

In [ ]:
import pyodbc
server = '####'
database = '####'
username = '######'
password = '######'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
with cnxn:
    with cnxn.cursor() as crs:
        string = ('select #####')
    crs.execute(string)

In [ ]:
metrics_df = pd.read_sql(sql=string,con=cnxn)

In [ ]:
metrics_df.rename({3:'attendance','':'baptisms','':'salvations'},axis='columns', inplace=True)

In [ ]:
metrics_df.columns.values[2] = 'attendance'
metrics_df.columns.values[3] = 'baptisms'
metrics_df.columns.values[4] = 'salvations'

In [ ]:
metrics_df.columns

In [ ]:
metrics_df

In [ ]:
metrics_df.shape

In [ ]:
df_by_year = df_by_year[df_by_year['campus']!='El Encuentro']

In [ ]:
df_by_year.shape

In [ ]:
results_df_merged = pd.merge(df_by_year,metrics_df,how='outer',left_on=['campus','Year'],right_on=['CampusName',"ServiceYear"])

In [ ]:
results_df_merged

In [ ]:
#let's clean up this dataframe, add columns with the differences and keep for exporting into excel
csv_totals_df = df_by_year.copy().replace(to_replace=['Dana Point Harbor','Manila'],value=['Dana Point','South Manila']).rename(columns={'attendance':'csv_attendance','baptisms':'csv_baptisms','salvations':'csv_salvations'}).fillna(0)
metrics_df = metrics_df.rename(columns={'attendance':'metrics_attendance','salvations':'metrics_salvations','baptisms':'metrics_baptisms'}).fillna(0)

In [ ]:
results_df = pd.merge(csv_totals_df,metrics_df,left_on=['campus','Year'],right_on=['CampusName','ServiceYear']).drop(axis=1,labels = ['CampusName','ServiceYear'])
results_df

In [ ]:
results_df['diff_attendance'] = results_df['csv_attendance']-results_df['metrics_attendance']
results_df['diff_baptisms'] = results_df['csv_baptisms']-results_df['metrics_baptisms']
results_df['diff_salvations'] = results_df['csv_salvations']-results_df['metrics_salvations']
results_df = results_df[(results_df['diff_attendance']!=0)|(results_df['diff_baptisms']!=0)|(results_df['diff_salvations']!=0)]
results_df

### Conclusion 2
- No difference between fact metrics and csv on baptisms and salvations
- Most of the differences are on 2019 which could be to the noise added lately to the data as part of tests
- Only Corona has some differences in data from  years other than 2019

In [ ]:
fact_metrics_yearly_total_errors = results_df.copy() #these are the errors df for yearly totals for the Fact Metrics Table

### Counting rows per Campus

In [ ]:
csv_df = df.copy() #copying the dataframe from the original load from csv file
csv_df.head()

In [ ]:
def count_campus_rows(df):
    result_df = df[df['Year'].isin([2017,2018,2019])].groupby(by=['campus','Year'],as_index=False)['attendance'].count()
    return result_df

In [ ]:
csv_rows_count = count_campus_rows(csv_df).rename(columns={'attendance':'csv_rows'}).replace(to_replace=['Dana Point Harbor','Manila'],value=['Dana Point','South Manila'])
csv_rows_count

In [ ]:
#let's now do this query on the Fact Metrics table on the warehouse
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
with cnxn:
    with cnxn.cursor() as crs:
        string = ('####')
    crs.execute(string)

In [ ]:
metrics_count_df = pd.read_sql(sql=string,con=cnxn)

In [ ]:
metrics_count_df

In [ ]:
csv_rows_count = csv_rows_count[csv_rows_count['campus']!='El Encuentro']

In [ ]:
metrics_count_df.shape

In [ ]:
csv_rows_count.shape

In [ ]:
count_comparisson_metrics_csv = pd.merge(csv_rows_count,metrics_count_df,how='inner',left_on=['campus','Year'],right_on=['CampusName','ServiceYear']).drop(['CampusName','ServiceYear'],axis=1).rename(columns={'':'metrics_rows'})
count_comparisson_metrics_csv

In [ ]:
count_comparisson_metrics_csv = count_comparisson_metrics_csv[count_comparisson_metrics_csv['metrics_rows']!=count_comparisson_metrics_csv['csv_rows']]

In [ ]:
count_comparisson_metrics_csv

## Comparing Calculations Vs CSV

I will start by the Monthyly Totals 

In [ ]:
#copying df already formatted from before
csv_totals = csv_totals_df

In [ ]:
#creating new dataframe from new query 
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
with cnxn:
    with cnxn.cursor() as crs:
        string = ('#####')
    crs.execute(string)
        


In [ ]:
monthly_total_df = pd.read_sql(sql=string,con=cnxn).fillna(0).rename(columns={'TotalAttendance':'summary_attendance','TotalBaptisms':'summary_baptisms','TotalSalvations':'summary_salvations'})
monthly_total_df

In [ ]:
#merging with csv data for comparisson
monthly_totals_results = pd.merge(csv_totals,monthly_total_df,left_on=['campus','Year'],right_on=['CampusName','Year']).drop(axis=1,labels=['CampusName'])
monthly_totals_results


In [ ]:
monthly_totals_results['diff_attendance'] = monthly_totals_results['csv_attendance']-monthly_totals_results['summary_attendance']
monthly_totals_results['diff_baptisms'] = monthly_totals_results['csv_baptisms']-monthly_totals_results['summary_baptisms']
monthly_totals_results['diff_salvations'] = monthly_totals_results['csv_salvations']-monthly_totals_results['summary_salvations']
monthly_totals_results

In [ ]:
monthlyTotal_vs_csv = monthly_totals_results[(monthly_totals_results['diff_attendance']!=0)|(monthly_totals_results['diff_salvations']!=0)|(monthly_totals_results['diff_baptisms']!=0)]
monthlyTotal_vs_csv

## __Comparing Monthly Averages Calculations Vs CSV__

In [ ]:
df[df['campus']=='Anaheim']

In [ ]:
#getting the number of weeks
grouped_df = df[df['Year'].isin([2017,2018,2019])].copy()
grouped_df = grouped_df.groupby(by=['campus','service_date','Year','Month'],as_index=False)[['attendance']].sum()
grouped_df = grouped_df[grouped_df['attendance']!=0]
grouped_df


In [ ]:
grouped_totals = grouped_df.groupby(by=['campus','Year','Month'],as_index=False).sum()
grouped_totals.shape

In [ ]:
grouped_df['dow'] = grouped_df['service_date'].dt.dayofweek
grouped_df
grouped_counts = grouped_df[grouped_df['dow']==6].groupby(by=['campus','Year','Month'],as_index=False)['service_date'].nunique()
grouped_counts.head()

In [ ]:
monthly_avg = grouped_totals.copy()
monthly_avg['month_weeks'] = grouped_counts
monthly_avg['month_avg'] = (monthly_avg['attendance']/monthly_avg['month_weeks']).astype('int32').round(0)
monthly_avg

In [ ]:
all_campuses_monthly_avg = monthly_avg.groupby(by=['Year','Month'],as_index=False).mean()
all_campuses_monthly_avg

## Exporticheck up dataframes into one single excel file to share with team

In [ ]:
writer = pd.ExcelWriter('metrics Vs csv.xlsx',engine='xlsxwriter')

In [ ]:
fact_metrics_yearly_total_errors.to_excel(writer,sheet_name='YearlyTotaldifferences')
count_comparisson_metrics_csv.to_excel(writer,sheet_name='RowCountDifferences')
monthlyTotal_vs_csv.to_excel(writer,sheet_name='MonthlyTotalCalculationErrors')
writer.save()